# Live Black Scholes Pricing Model Using JAX

##### Zach Petko - 3/13/2025 

---

### Personal Background and Motivations for This Project
The motivation behind creating this Black-Scholes pricing model stems from my deep interest in the intersection of finance and technology. I’ve always been fascinated by how mathematical models can be applied to real-world financial markets to manage risk and make informed investment decisions. As a student eager to explore quantitative finance, I wanted to gain hands-on experience by implementing one of the most fundamental models in options pricing. The opportunity to work with live market data and apply advanced computational tools like JAX made this project especially appealing. By building this model, I aimed to enhance my understanding of option pricing theory, improve my programming skills, and explore how modern computing techniques can optimize financial calculations. This project not only allows me to apply theoretical knowledge to practical scenarios but also prepares me for a future career in quantitative analysis or financial engineering, where I hope to contribute to the evolution of financial modeling and risk management strategies. Thank you for viewing my work and I hope you enjoy!



---

## Table of Contents

### 1. Introduction
Explains the background of the Black-Scholes model and the motivations for working on this project.

### 2. Data Collection

Details the process of gathering real-time data including options, stock, and treasury information from Yahoo Finance.

### 3. Black-Scholes Model

Describes the Black-Scholes pricing formulas and their implementation. The model created for this project is compared to existing implementations in python from the BlackScholes library.


---

## 1. Introduction

Accurately pricing options is a critical component of modern financial markets, enabling traders and investors to manage risk and make informed decisions. The **Black-Scholes model**, introduced in 1973 by Fischer Black, Myron Scholes, and Robert Merton, is one of the most widely used mathematical models for option pricing. It calculates the theoretical price of European-style options by considering factors such as the underlying asset price, strike price, time to expiration, volatility, risk-free interest rate, and dividends.

This project implements a live Black-Scholes pricing model using **JAX**, a high-performance library for automatic differentiation and numerical computing. The model is designed to price call options in real-time using market data from Yahoo Finance, and its performance is compared to an existing Black-Scholes implementation.

---

## 2. Data Collection

### Install Packages

In [126]:
#run in terminal to install packages: pip install package 

import yfinance as yf # yahoo finance 
import pandas as pd # pandas
import jax.numpy as jnp # jax numpy
from jax import grad # gradiant from jax
from jax.scipy.stats import norm as jnorm # norm density from jax

import blackscholes as bs # existing implementation to check work



### Load Options Data

Assign a valid stock ticker to ticker_input then run the code block

In [127]:
#input stock ticker
ticker_input = "AAPL" 


#create yf ticker for inputed ticker
ticker = yf.Ticker(ticker_input) 

#get all option expirations
expirations = ticker.options 

#create empty dataframes to add to with the loop
all_options = pd.DataFrame()
all_calls = pd.DataFrame()
all_puts = pd.DataFrame()

#loop through expirations and get option chain
for exp in expirations: 
    options_chain = ticker.option_chain(exp)
    calls = options_chain.calls
    puts = options_chain.puts
    calls['expiration'] = exp
    puts['expiration'] = exp
    calls['optionType'] = 'Call'
    puts['optionType'] = 'Put'

    #concatinate calls and puts for each expiration
    all_options = pd.concat([all_options, calls, puts], ignore_index=True)
    all_calls = pd.concat([all_calls, calls], ignore_index=True)
    all_puts = pd.concat([all_puts, puts], ignore_index=True)
#display options table
display(all_options)


,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,expiration,optionType
0,AAPL250314C00100000,2025-03-05 14:42:20+00:00,100.0,133.96,109.40,110.05,0.000000,0.000000,1.0,0,5.281253,True,REGULAR,USD,2025-03-14,Call
1,AAPL250314C00120000,2025-03-12 15:46:51+00:00,120.0,97.20,89.45,90.05,0.000000,0.000000,2.0,7,4.265630,True,REGULAR,USD,2025-03-14,Call
2,AAPL250314C00130000,2025-03-11 13:30:08+00:00,130.0,93.32,79.45,80.05,0.000000,0.000000,1.0,1,3.703126,True,REGULAR,USD,2025-03-14,Call
3,AAPL250314C00140000,2025-03-12 15:15:31+00:00,140.0,75.22,69.45,70.05,0.000000,0.000000,36.0,38,3.171877,True,REGULAR,USD,2025-03-14,Call
4,AAPL250314C00145000,2025-03-13 19:07:44+00:00,145.0,66.45,64.45,65.05,-11.920006,-15.209909,1.0,5,2.921878,True,REGULAR,USD,2025-03-14,Call
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017,AAPL270617P00410000,2025-02-14 14:39:31+00:00,410.0,166.68,198.00,203.00,0.000000,0.000000,3.0,0,0.277748,True,REGULAR,USD,2027-06-17,Put
2018,AAPL270617P00420000,2025-02-13 14:44:10+00:00,420.0,183.10,208.00,213.00,0.000000,0.000000,3.0,0,0.285011,True,REGULAR,USD,2027-06-17,Put
2019,AAPL270617P00430000,2025-02-12 14:45:18+00:00,430.0,198.33,218.00,223.00,0.000000,0.000000,3.0,0,0.292060,True,REGULAR,USD,2027-06-17,Put
2020,AAPL270617P00440000,2025-03-03 14:30:10+00:00,440.0,197.65,228.00,233.00,0.000000,0.000000,1.0,0,0.298896,True,REGULAR,USD,2027-06-17,Put


### Filter Call and Put Dataframes

In [148]:
#select columns
all = all_options[['bid', 'ask', 'strike', 'impliedVolatility', 'lastPrice', 'expiration', 'optionType']].copy()

#add a ticker column
all['ticker'] = ticker_input

#create time till expiration column
today_date = pd.to_datetime("today").normalize()
all['expiration'] = pd.to_datetime(all['expiration'])
all['yearsToMaturity'] = ((all['expiration'] - today_date).dt.days)/365 #time to expiration is in years

#filter out options with 0 values
all = all[all['lastPrice']>0]
all = all[all['strike']>0]
all = all[all['impliedVolatility']>0]
all = all[all['yearsToMaturity']>0]

#split to calls and puts datasets
calls = all[all['optionType']=='Call']
puts = all[all['optionType']=='Put']



display("Call Options: ", calls, "Put Options:", puts)


'Call Options: '

,bid,ask,strike,impliedVolatility,lastPrice,expiration,optionType,ticker,yearsToMaturity
0,109.40,110.05,100.0,5.281253,133.96,2025-03-14,Call,AAPL,0.002740
1,89.45,90.05,120.0,4.265630,97.20,2025-03-14,Call,AAPL,0.002740
2,79.45,80.05,130.0,3.703126,93.32,2025-03-14,Call,AAPL,0.002740
3,69.45,70.05,140.0,3.171877,75.22,2025-03-14,Call,AAPL,0.002740
4,64.45,65.05,145.0,2.921878,66.45,2025-03-14,Call,AAPL,0.002740
...,...,...,...,...,...,...,...,...,...
1964,2.26,2.88,410.0,0.279975,3.20,2027-06-17,Call,AAPL,2.263014
1965,2.15,2.61,420.0,0.281440,2.40,2027-06-17,Call,AAPL,2.263014
1966,1.92,2.46,430.0,0.284980,2.35,2027-06-17,Call,AAPL,2.263014
1967,1.78,2.15,440.0,0.284126,1.95,2027-06-17,Call,AAPL,2.263014


'Put Options:'

,bid,ask,strike,impliedVolatility,lastPrice,expiration,optionType,ticker,yearsToMaturity
56,0.0,0.01,100.0,4.375005,0.01,2025-03-14,Put,AAPL,0.002740
57,0.0,0.01,110.0,3.812500,0.01,2025-03-14,Put,AAPL,0.002740
58,0.0,0.01,120.0,3.312502,0.01,2025-03-14,Put,AAPL,0.002740
59,0.0,0.01,130.0,2.875003,0.01,2025-03-14,Put,AAPL,0.002740
60,0.0,0.01,140.0,2.437504,0.01,2025-03-14,Put,AAPL,0.002740
...,...,...,...,...,...,...,...,...,...
2017,198.0,203.00,410.0,0.277748,166.68,2027-06-17,Put,AAPL,2.263014
2018,208.0,213.00,420.0,0.285011,183.10,2027-06-17,Put,AAPL,2.263014
2019,218.0,223.00,430.0,0.292060,198.33,2027-06-17,Put,AAPL,2.263014
2020,228.0,233.00,440.0,0.298896,197.65,2027-06-17,Put,AAPL,2.263014


### Load Stock Price and Yield

In [ ]:
#stock price
stockPrice = ticker.fast_info['lastPrice']

print(f"{ticker_input} current stock price is : ${stockPrice}")

#dividendYield
dividendYield = ticker.info.get('dividendYield')
print(f"{ticker_input} current Dividend Yield is : {dividendYield}")

AAPL current stock price is : $209.67999267578125
AAPL current Dividend Yield is : 0.46


### Load Risk Free Rate
Input appropriate treasury yield matching time to expiration

In [130]:
#^IRX: 3-month Treasury bill
#^FVX: 5-year Treasury note
#^TNX: 10-year Treasury note
#^TYX: 30-year Treasury bond

#import from yahoo finance
riskFreeRate = yf.Ticker("^TNX").history(period="1d")['Close'].iloc[-1] / 100

print(f"Risk Free Rate: {riskFreeRate}")


Risk Free Rate: 0.042740001678466796


---

## 3. Black-Scholes Model 

### Call Option Price:

$$
C = S e^{-q T} N(d_1) - K e^{-r T} N(d_2)
$$

### Put Option Price:

$$
P = K e^{-r T} N(-d_2) - S e^{-q T} N(-d_1)
$$

- \( S \) = Current stock price  
- \( K \) = Strike price  
- \( T \) = Time to maturity (in years)  
- \( r \) = Risk-free interest rate (annual)
- \( $\sigma$ \) = Volatility of the stock (SD of returns)  
- \( q \) = Dividend yield  
- \( N(x) \) = Standard Normal Cumulative distribution function (CDF)

$$
d_1 = \frac{\ln\left(\frac{S}{K}\right) + \left(r - q + \frac{\sigma^2}{2}\right) T}{\sigma \sqrt{T}}
$$

$$
d_2 = d_1 - \sigma \sqrt{T}
$$


### Black-Scholes Function

In [131]:
def black_scholes(S, K, T, r, sigma, q=0, opt_type='call'):
    d1 = (jnp.log(S / K) + (r - q + 0.5 * sigma **2 ) * T ) / (sigma * jnp.sqrt(T))
    d2 = d1 - sigma * jnp.sqrt(T)
    if opt_type == 'call':
        call = S * jnp.exp(-q * T) * jnorm.cdf(d1, 0, 1) - K * jnp.exp(-r * T) * jnorm.cdf(d2, 0, 1)
        return float(call)
    else:
        put = K * jnp.exp(-r * T) * jnorm.cdf(-d2, 0, 1) - S * jnp.exp(-q * T) * jnorm.cdf(-d1, 0, 1)
        return float(put)

In [132]:
calls['impliedVolatility'].describe()

count    1044.000000
mean        0.728420
std         1.265998
min         0.000010
25%         0.309234
50%         0.418188
75%         0.647663
max        20.128910
Name: impliedVolatility, dtype: float64

In [150]:
# Our Model

#create empty list to collect values
estCallPrices = []  

S = stockPrice
r = riskFreeRate
q=dividendYield

for i in range(len(calls)):
    K = calls.iloc[i]['strike']
    T = calls.iloc[i]['yearsToMaturity']
    sigma = calls.iloc[i]['impliedVolatility']
    
    #calculate price using black scholes and append to estCallPrices
    price = black_scholes(S, K, T, r, sigma, q, opt_type='call')  
    estCallPrices.append(price)  

# Convert list to DataFrame at the end
estCallPrices = pd.Series(estCallPrices)



# Existing library

#create empty list to collect values
estCallPrices2 = []  

S = stockPrice
r = riskFreeRate
q=dividendYield

for i in range(len(calls)):
    K = calls.iloc[i]['strike']
    T = calls.iloc[i]['yearsToMaturity']
    sigma = calls.iloc[i]['impliedVolatility']
    
    #calculate price using black scholes and append to estCallPrices
    price = bs.BlackScholesCall(S=S, K=K, T=T, r=r, sigma=sigma, q=q).price() 
    estCallPrices2.append(price)  

# Convert list to DataFrame at the end
estCallPrices2 = pd.Series(estCallPrices2)




#Combine all in a df to compare
compEstCallPrices = pd.DataFrame({'Real Call Price' : calls['lastPrice'], 
                                 'Our Model Call Price' : estCallPrices,
                                  'Existing Model Call Price' : estCallPrices2})
display(compEstCallPrices.head(50))




,Real Call Price,Our Model Call Price,Existing Model Call Price
0,133.96,109.473335,109.473352
1,97.20,89.501724,89.501739
2,93.32,79.503235,79.503250
3,75.22,69.503540,69.503547
4,66.45,64.504257,64.504278
5,78.17,59.505142,59.505164
6,62.95,54.506393,54.506399
7,51.07,49.506210,49.506235
8,48.99,44.506638,44.506639
9,41.13,39.534805,39.534816
